<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/NN/Lab%201/Lab_1_Small_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Αϊδίνης Γιώργος 03116031

Κολιός Παναγιώτης 03116100

---

Ομάδα M.B.8

---


#Lab 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση - Μικρό Dataset (S11 - Quality Assessment of Digital Colposcopies)

Αρχικά ενημερώνουμε τις βιβλιοθήκες που θα χρησιμοποιήσουμε.



In [1]:
!pip install --upgrade pip #upgrade pip package installer
!pip install scikit-learn --upgrade #upgrade scikit-learn package
!pip install numpy --upgrade #upgrade numpy package
!pip install pandas --upgrade #--upgrade #upgrade pandas package

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 7.0MB 5.0MB/s 
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.17.4)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.25.3)


In [0]:
from urllib.request import urlretrieve
import os
import zipfile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")


In [3]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/00384/Quality%20Assessment%20-%20Digital%20Colposcopy.zip","QADC.zip")
print("All the files are downloaded")

Download file... QADC.zip ...
File downloaded
All the files are downloaded


In [4]:
!ls

QADC.zip  sample_data


In [0]:
!unzip -q "QADC.zip"

In [6]:
!ls

 QADC.zip  'Quality Assessment - Digital Colposcopy'   sample_data


Rename directory for better handling



In [0]:
!mv Quality\ Assessment\ -\ Digital\ Colposcopy QADC

Remove attributes line for the 2nd and 3rd files:

In [0]:
with open("QADC/hinselmann.csv",'r') as f:
    with open("QADC/hinselmann1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

In [0]:
with open("QADC/schiller.csv",'r') as f:
    with open("QADC/schiller1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

create one big .csv

In [0]:
!cat QADC/green.csv QADC/hinselmann1.csv QADC/schiller1.csv > all.csv

We needed to concatenate the 3 .csv files && remove the category names from the second and third files gia na min exoume asxeta instances
We also need to remove the attribute of the experts, apart from the concensus

In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("all.csv")
df = df.drop(df.columns[[62, 63, 64, 65, 66, 67]], axis=1)

Let's print number, names and types of attributes

In [12]:
print ("We have ",len(df.columns), " attributes.")
for i in range(0, len(df.columns)):
    print(i+1, ": \t", df.columns[i], "type: ", df.dtypes[df.columns[i]])

We have  63  attributes.
1 : 	 cervix_area type:  float64
2 : 	 os_area type:  float64
3 : 	 walls_area type:  float64
4 : 	 speculum_area type:  float64
5 : 	 artifacts_area type:  float64
6 : 	 cervix_artifacts_area type:  float64
7 : 	 os_artifacts_area type:  float64
8 : 	 walls_artifacts_area type:  float64
9 : 	 speculum_artifacts_area type:  float64
10 : 	 cervix_specularities_area type:  float64
11 : 	 os_specularities_area type:  float64
12 : 	 walls_specularities_area type:  float64
13 : 	 speculum_specularities_area type:  float64
14 : 	 specularities_area type:  float64
15 : 	 area_h_max_diff type:  float64
16 : 	 rgb_cervix_r_mean type:  float64
17 : 	 rgb_cervix_r_std type:  float64
18 : 	 rgb_cervix_r_mean_minus_std type:  float64
19 : 	 rgb_cervix_r_mean_plus_std type:  float64
20 : 	 rgb_cervix_g_mean type:  float64
21 : 	 rgb_cervix_g_std type:  float64
22 : 	 rgb_cervix_g_mean_minus_std type:  float64
23 : 	 rgb_cervix_g_mean_plus_std type:  float64
24 : 	 rgb_cervix

We need to count the occurences of each label (healthy or unhealthy):

In [13]:
labels_df = df.iloc[:, [62]]
labels = labels_df.values.reshape(287,)
positive = 0
negative = 0
for i in range(0, len(labels)):
    positive += labels[i]==1
    negative += labels[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  216  negative:  71


Train - Test split

In [0]:
from sklearn.model_selection import train_test_split

train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.2)

Undersampling - Oversampling

In [22]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
train_resampled, train_labels_resampled = ros.fit_sample(train,train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [29]:
positive = 0
negative = 0
for i in range(0, len(train_labels_resampled)):
    positive += train_labels_resampled[i]==1
    negative += train_labels_resampled[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  176  negative:  176
